In [1]:
# collections是Python内建的一个集合模块，提供了许多有用的集合类。

# namedtuple
# 我们知道tuple可以表示不变集合，例如，一个点的二维坐标就可以表示成：

p = (1, 2)
# 但是，看到(1, 2)，很难看出这个tuple是用来表示一个坐标的。
# 定义一个class又小题大做了，这时，namedtuple就派上了用场：

In [12]:
from collections import namedtuple
Point = namedtuple('Point', ['x', 'y'])
p = Point(1, 2)
print(p.x)
print(p.y)

1
2


In [5]:
# namedtuple是一个函数，它用来创建一个自定义的tuple对象，
# 并且规定了tuple元素的个数，并可以用属性而不是索引来引用tuple的某个元素。

# 这样一来，我们用namedtuple可以很方便地定义一种数据类型，它具备tuple的不变性，又可以根据属性来引用，使用十分方便。
# 可以验证创建的Point对象是tuple的一种子类：

print(isinstance(p, Point))
print(isinstance(p, tuple))

True
True


In [6]:
# 类似的，如果要用坐标和半径表示一个圆，也可以用namedtuple定义：
Circle = namedtuple('Circle', ['x', 'y', 'r'])

In [13]:
# add
# ✅ 推荐：类型名与变量名一致
Point = namedtuple('Point', ['x', 'y'])

# ❌ 不推荐：类型名与变量名不一致
MyPoint = namedtuple('Point', ['x', 'y'])  # 混淆！

# 混乱的例子（不要这样做！）
WeirdTuple = namedtuple('StrangeName', ['a', 'b'])
wt = WeirdTuple(1, 2)

print(wt)  # StrangeName(a=1, b=2) - 显示的是 'StrangeName'
print(type(wt).__name__)  # 'StrangeName' - 类名是 StrangeName
print(WeirdTuple.__name__)  # 'StrangeName' - 变量名是 WeirdTuple，但类名是 StrangeName

StrangeName(a=1, b=2)
StrangeName
StrangeName


In [ ]:
# deque

# 使用list存储数据时，按索引访问元素很快，但是插入和删除元素就很慢了，
# 因为list是线性存储，数据量大的时候，插入和删除效率很低。
# deque是为了高效实现插入和删除操作的双向列表，适合用于队列和栈：

from collections import deque
q = deque(['a', 'b', 'c'])
q.append('x')
q.appendleft('y')
print(q)

# deque除了实现list的append()和pop()外，
# 还支持appendleft()和popleft()，这样就可以非常高效地往头部添加或删除元素。

deque(['y', 'a', 'b', 'c', 'x'])


In [ ]:
# defaultdict

# 使用dict时，如果引用的Key不存在，就会抛出KeyError。
# 如果希望key不存在时，返回一个默认值，就可以用defaultdict：

from collections import defaultdict
dd = defaultdict(lambda: 'N/A')
dd['key1'] = 'abc'
print(dd['key1']) # key1存在
print(dd['key2'])

# 注意默认值是调用函数返回的，而函数在创建defaultdict对象时传入。
# 除了在Key不存在时返回默认值，defaultdict的其他行为跟dict是完全一样的。

abc
N/A


In [18]:
# OrderedDict
# 使用dict时，Key是无序的。在对dict做迭代时，我们无法确定Key的顺序。
# 如果要保持Key的顺序，可以用OrderedDict：

from collections import OrderedDict
d = dict([('a', 1), ('b', 2), ('c', 3)])
print(d) # dict的Key是无序的

od = OrderedDict([('a', 1), ('b', 2), ('c', 3)])
print(od) # OrderedDict的Key是有序的

{'a': 1, 'b': 2, 'c': 3}
OrderedDict([('a', 1), ('b', 2), ('c', 3)])


In [19]:
# 注意，OrderedDict的Key会按照插入的顺序排列，不是Key本身排序：

od = OrderedDict()
od['z'] = 1
od['y'] = 2
od['x'] = 3
list(od.keys()) # 按照插入的Key的顺序返回

['z', 'y', 'x']

In [21]:
# OrderedDict可以实现一个FIFO（先进先出）的dict，当容量超出限制时，先删除最早添加的Key：
from collections import OrderedDict

class LastUpdatedOrderedDict(OrderedDict):

    def __init__(self, capacity):
        super().__init__()
        self._capacity = capacity

    def __setitem__(self, key, value):
        containsKey = 1 if key in self else 0
        if len(self) - containsKey >= self._capacity:
            last = self.popitem(last=False)
            print('remove:', last)
        if containsKey:
            del self[key]
            print('set:', (key, value))
        else:
            print('add:', (key, value))
        OrderedDict.__setitem__(self, key, value)

In [ ]:
# ChainMap
# ChainMap可以把一组dict串起来并组成一个逻辑上的dict。
# ChainMap本身也是一个dict，但是查找的时候，会按照顺序在内部的dict依次查找。

# 什么时候使用ChainMap最合适？
# 举个例子：应用程序往往都需要传入参数，参数可以通过命令行传入，可以通过环境变量传入，还可以有默认参数。
# 我们可以用ChainMap实现参数的优先级查找，即先查命令行参数，如果没有传入，再查环境变量，如果没有，就使用默认参数。

# 下面的代码演示了如何查找user和color这两个参数：

from collections import ChainMap
import os, argparse

# 构造缺省参数:
# 运行Chainmap。py
defaults = {
    'color': 'red',
    'user': 'guest'
}

# 构造命令行参数:
parser = argparse.ArgumentParser()
parser.add_argument('-u', '--user')
parser.add_argument('-c', '--color')
namespace = parser.parse_args()
command_line_args = { k: v for k, v in vars(namespace).items() if v }

# 组合成ChainMap:
combined = ChainMap(command_line_args, os.environ, defaults)

# 打印参数:
print('color=%s' % combined['color'])
print('user=%s' % combined['user'])

usage: ipykernel_launcher.py [-h] [-u USER] [-c COLOR]
ipykernel_launcher.py: error: unrecognized arguments: --f=/run/user/1000/jupyter/runtime/kernel-v30cd2ba20166373fa04ea456874c41df8ab935697.json


SystemExit: 2

/home/qwx/miniforge3/envs/personal/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [24]:
# 没有任何参数时，打印出默认参数：
# python3 use_chainmap.py
# 当传入命令行参数时，优先使用命令行参数：
# python3 use_chainmap.py -u bob
# 同时传入命令行参数和环境变量，命令行参数的优先级较高：
# user=admin color=green python3 use_chainmap.py -u bob 

In [ ]:
# Counter
# Counter是一个简单的计数器，例如，统计字符出现的个数：
from collections import Counter
c = Counter('programming')
print(c)

for ch in 'programming':
    c[ch] = c[ch] + 1
print(c) # 手动计数

c.update('hello') # 也可以一次性update
print(c)
# Counter实际上也是dict的一个子类，上面的结果可以看出每个字符出现的次数。

Counter({'r': 2, 'g': 2, 'm': 2, 'p': 1, 'o': 1, 'a': 1, 'i': 1, 'n': 1})
Counter({'r': 4, 'g': 4, 'm': 4, 'p': 2, 'o': 2, 'a': 2, 'i': 2, 'n': 2})
Counter({'r': 4, 'g': 4, 'm': 4, 'o': 3, 'p': 2, 'a': 2, 'i': 2, 'n': 2, 'l': 2, 'h': 1, 'e': 1})
